In [ ]:
!pip install tflite-support

In [ ]:
!mkdir TensorFlow 
%cd TensorFlow
!git clone https://github.com/tensorflow/models.git
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!pip install cython
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
%cd ../../..
!git clone https://github.com/Hyuto/rangrang-ML.git
!cp -r rangrang-ML/workspace TensorFlow
%cd rangrang-ML
!git checkout tensorflow-od
%cd ..
!cp -r rangrang-ML/workspace TensorFlow
%cd TensorFlow/workspace/training_demo
!mkdir exported-models

In [ ]:
# Generate tfrecord
!python scripts/preprocessing/generate_tfrecord.py -x images/train -l annotations/label_map.pbtxt -o annotations/train.record
!python scripts/preprocessing/generate_tfrecord.py -x images/test -l annotations/label_map.pbtxt -o annotations//test.record

In [ ]:
!python model_main_tf2.py --model_dir=models/model_aku --pipeline_config_path=models/model_aku/pipeline.config

In [ ]:
%cd ../../..

In [ ]:
!python TensorFlow/models/research/object_detection/export_tflite_graph_tf2.py \
        --pipeline_config_path TensorFlow/workspace/training_demo/models/model_aku/pipeline.config \
        --trained_checkpoint_dir TensorFlow/workspace/training_demo/models/model_aku/ \
        --output_directory TensorFlow/workspace/training_demo/exported-models

In [ ]:
import tensorflow as tf
import numpy as np

def representative_dataset():
    for _ in range(1):
      data = np.random.rand(1, 320, 320, 3)
      yield [data.astype(np.float32)]

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('TensorFlow/workspace/training_demo/exported-models/saved_model') 
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8,
                                       tf.lite.OpsSet.TFLITE_BUILTINS]
converter.representative_dataset = representative_dataset
tflite_model = converter.convert()

# Save the model.
with open('detect.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
LABELS = [
    'axe'
]

with open('labelmap.txt', 'w') as w:
  w.write('\n'.join(LABELS))

In [ ]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata

ObjectDetectorWriter = object_detector.MetadataWriter
MODEL_PATH = "detect.tflite"
LABEL_FILE = "labelmap.txt"
EXPORT_PATH = "detect.tflite"

writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(MODEL_PATH), [127.5], [127.5], [LABEL_FILE])
writer_utils.save_file(writer.populate(), EXPORT_PATH)

# Verify the populated metadata and associated files.
displayer = metadata.MetadataDisplayer.with_model_file(_SAVE_TO_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

In [ ]:
from google.colab import files

files.download('detect.tflite')